# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
session.execute("create keyspace if not exists sparkify with replication = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }")

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace("sparkify")

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "create table if not exists session_library "
query = query + "(sessionid int, iteminsession int, artist text, song text, length float, primary key(sessionid, iteminsession))"
session.execute(query)
                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "insert into session_library (sessionid, iteminsession, artist, song, length) "
        query = query + "values (%s,%s,%s,%s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (pd.to_numeric(line[8],downcast='integer'), pd.to_numeric(line[3],downcast='integer'), line[0], line[9], pd.to_numeric(line[5],downcast='float')))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
## Add in the SELECT statement to verify the data was entered into the table
query = "select artist,song,length from session_library where sessionid=338 and iteminsession=4"
rows = session.execute(query)
for row in rows:
    print(row)
#    print(row.sessionid, row.iteminsession, row.artist, row.song, row.length)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.3070068359375)


### Explanation
Our **primary key** includes both the ``sessionId`` and ``itemInSession``, because that makes the table's partition key so our query can run efficiently

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [13]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

#create table
query = "create table if not exists user_library "
query = query + "(userid int, sessionid int, iteminsession int, artist text, song text, firstname text, lastname text, primary key((userid, sessionid), itemInSession))"
session.execute(query)

#insert
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into user_library (userid, sessionid, iteminsession, artist, song, firstname, lastname) "
        query = query + "values (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (pd.to_numeric(line[10],downcast='integer'), pd.to_numeric(line[8],downcast='integer'), pd.to_numeric(line[3],downcast='integer'), line[0], line[9], line[1], line[4]))

#select
query = "select artist,song,firstname,lastname from user_library where userid=10 and sessionid=182"
rows = session.execute(query)
for row in rows:
#    print(row)
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Explanation
Our **primary key** includes both the ``userId`` and ``sessionId``, because that makes the table's partition key so our query can run efficiently. It also includes the ``itemInSession`` as a clustering column on which we achieve the proper ordering.

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#create table: we need to include the userid as part of the key too, otherwise the end result would be only 1 (or less) item
query = "create table if not exists song_library "
query = query + "(song text, userid int, firstname text, lastname text, primary key(song, userid))"
session.execute(query)

#insert
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into song_library (song, userid, firstname, lastname) "
        query = query + "values (%s,%s,%s,%s)"
        session.execute(query, (line[9], pd.to_numeric(line[10],downcast='integer'), line[1], line[4]))

#select
query = "select firstname,lastname from song_library where song='All Hands Against His Own'"
rows = session.execute(query)
for row in rows:
#    print(row)
    print(row.firstname, row.lastname)

                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Explanation
Here we had to add both the ``song`` and ``userId`` to the **primary key**, otherwise the end result would have been only 1 (or less) item in case we base the table only on the ``song``. 

### Drop the tables before closing out the sessions

In [34]:
## Drop the table before closing out the sessions
query = "drop table session_library"
session.execute(query)
query = "drop table user_library"
session.execute(query)
query = "drop table song_library"
session.execute(query)

### Close the session and cluster connection¶

In [35]:
session.shutdown()
cluster.shutdown()